In [1]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np

# Machine Learning Libraries
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# Processing of a Sales Forecasting MLP & Random Forest Model

## Pre-Process 

In [3]:
store = pd.read_csv("store.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\2382029950.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("train.csv")


Three data sets as described. Train has data for the historic period, and test only has for the period when the forcast should be made.

In [4]:
working_file=pd.merge(train, store, on='Store')
test_file=pd.merge(test, store, on='Store')


In [5]:
working_file['StateHoliday'].replace({'0': 0}, inplace=True)
test_file['StateHoliday'].replace({'0': 0}, inplace=True)

C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\2180710356.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  working_file['StateHoliday'].replace({'0': 0}, inplace=True)
C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\2180710356.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [6]:
Store_open_data= working_file[working_file["Open"]>=1]
Store_open_test= test_file[test_file["Open"]>=1]

## Start here with Store_open_data, and Store_open_test

### Convert time data 

In [7]:
Store_open_test['Date'] = pd.to_datetime(Store_open_test.Date)

Store_open_test['Year'] = Store_open_test.Date.dt.year
Store_open_test['Month'] = Store_open_test.Date.dt.month
Store_open_test['Day'] = Store_open_test.Date.dt.day

C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\4006022767.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Store_open_test['Date'] = pd.to_datetime(Store_open_test.Date)
C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\4006022767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Store_open_test['Date'] = pd.to_datetime(Store_open_test.Date)
C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\4006022767.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [8]:
Store_open_data['Date'] = pd.to_datetime(Store_open_data.Date)

Store_open_data['Year'] = Store_open_data.Date.dt.year
Store_open_data['Month'] = Store_open_data.Date.dt.month
Store_open_data['Day'] = Store_open_data.Date.dt.day

C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\703457712.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Store_open_data['Date'] = pd.to_datetime(Store_open_data.Date)
C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\703457712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Store_open_data['Date'] = pd.to_datetime(Store_open_data.Date)
C:\Users\ashto\AppData\Local\Temp\ipykernel_25840\703457712.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [10]:
Store_open_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,620.0,9.0,2009.0,0,NaN,NaN,NaN,2015,7,31
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,29910.0,4.0,2015.0,0,NaN,NaN,NaN,2015,7,31


### Check Missing Values

In [8]:
missing = Store_open_data.isnull().sum()
missing.sort_values(ascending=False)

PromoInterval                423307
Promo2SinceYear              423307
Promo2SinceWeek              423307
CompetitionOpenSinceYear     268619
CompetitionOpenSinceMonth    268619
CompetitionDistance            2186
Store                             0
Month                             0
Year                              0
Promo2                            0
Assortment                        0
DayOfWeek                         0
StoreType                         0
SchoolHoliday                     0
StateHoliday                      0
Promo                             0
Open                              0
Customers                         0
Sales                             0
Date                              0
Day                               0
dtype: int64

We see missing values but not for any of the features we plan to use.

### Select training variables and catagorize variables for encoding

In [9]:
#Colums for training
input_cols = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'StoreType', 'Assortment', 'Promo2', 'Day', 'Month', 'Year']
#Target Columns
target_col = 'Sales'
#Sort into training inputs and targets 
train_inputs = Store_open_data[input_cols].copy()
train_targets = Store_open_data[target_col].copy()
#same as above but not target values needed
test_inputs = Store_open_test[input_cols].copy()
#Used to sort which colums will be scaled
num_cols = ['Store', 'DayOfWeek', 'Day', 'Month', 'Year']
cat_cols = ['DayOfWeek', 'Promo', 'StoreType', 'Assortment', 'Promo2']

In [17]:
train_inputs.head()

,Store,DayOfWeek,Promo,StateHoliday,StoreType,Assortment,Promo2,Day,Month,Year
0,1,5,1,0,c,a,0,31,7,2015
1,2,5,1,0,a,a,1,31,7,2015
2,3,5,1,0,a,a,1,31,7,2015
3,4,5,1,0,c,c,0,31,7,2015
4,5,5,1,0,a,a,0,31,7,2015


### Encode 

These variables are fed directly into the grid and random search functions below.

In [10]:
# Encoding
# get_dummies provides onehot coding
train_inputs = pd.get_dummies(train_inputs)
#val_inputs = pd.get_dummies(val_inputs, drop_first=False)
test_inputs = pd.get_dummies(test_inputs)

# Drop the columns that are not in test dataset
train_inputs = train_inputs.drop(['StateHoliday_0', 'StateHoliday_c'], axis=1)
test_inputs = test_inputs.drop(['StateHoliday_0'], axis=1)


In [27]:
train_inputs.head()

,Store,DayOfWeek,Promo,Promo2,Day,Month,Year,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
0,1,5,1,0,31,7,2015,True,False,False,False,False,False,True,False,True,False,False
1,2,5,1,1,31,7,2015,True,False,False,False,True,False,False,False,True,False,False
2,3,5,1,1,31,7,2015,True,False,False,False,True,False,False,False,True,False,False
3,4,5,1,0,31,7,2015,True,False,False,False,False,False,True,False,False,False,True
4,5,5,1,0,31,7,2015,True,False,False,False,True,False,False,False,True,False,False


### Split Data For Single Run Models

This is only used for the single run training, not for the CV training 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    train_inputs, train_targets, test_size=0.20, random_state=42)

### Scale Used for Single Run Models

In [20]:
#Train our MinMaxScaler on the colums we want to scale
scaler = MinMaxScaler().fit(X_train[num_cols])

# Now use the scaler to scale all of the colums we have trained it on.
X_train[num_cols] = scaler.transform(X_train[num_cols])
#val_inputs[num_cols] = scaler.transform(val_inputs[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

### RMSE function

Only used for the single split tests

In [23]:
def rmspe(y_true, y_pred):
   
    # Check lenght is equal
    assert len(y_true) == len(y_pred)
    
    # Percentage error
    percentage_error = (y_true - y_pred) / y_true
    
    # Remove null values
    percentage_error[y_true == 0] = 0
    
    # Square the percentage errors
    squared_percentage_error = percentage_error ** 2
    
    # Compute the mean of the squared percentage errors
    mean_squared_percentage_error = np.mean(squared_percentage_error)
    
    # Compute the square root of the mean squared percentage error
    rmspe = np.sqrt(mean_squared_percentage_error)
    
    return rmspe # Convert to percentage

# Random Forest Model 

## Single split and model fit 

This is a simple single run model used to test training time for different parameters. The score for this model was not used to evaluate the model overall.

In [20]:
# Train function 
random_forest_model = RandomForestRegressor(random_state=18,
                                            n_estimators= 300,
                                            min_samples_split= 3,
                                            min_samples_leaf= 1,
                                            max_depth= None, 
                                            bootstrap= True)

random_forest_model.fit(X_train,y_train)

RandomForestRegressor(min_samples_split=3, n_estimators=300, random_state=18)

### RMSE and RMPSE for Simple Model

In [ ]:
train_preds = random_forest_model.predict(X_train)
test_preds = random_forest_model.predict(X_test)

    # Get RMSE
train_rmse = np.round(root_mean_squared_error(y_train, train_preds, squared=False), 5)
val_rmse = np.round(root_mean_squared_error(y_test, test_preds, squared=False), 5)

    # Get RMSPE
train_rmspe = np.round(rmspe(y_train, train_preds), 5)
val_rmspe = np.round(rmspe(y_test, test_preds), 5)


print(f"Train RMSE: {train_rmse}")
print(f"Val RMSE: {val_rmse}")
print()
print(f"Train RMSPE: {train_rmspe}")
print(f"Val RMSPE: {val_rmspe}")

### Results

Model performance record:

Standard settings: 
Train RMSE: 402.78114
Val RMSE: 1072.6163

Train RMSPE: 0.06871
Val RMSPE: 0.30442

Random Search optomized settings:

Random search settings: random_state=18,
                                            n_estimators= 300,
                                            min_samples_split= 3,
                                            min_samples_leaf= 1,
                                            max_depth= None, 
                                            bootstrap= True)

Train RMSE: 461.21775
Val RMSE: 1062.12101

Train RMSPE: 0.08619
Val RMSPE: 0.30292

## Cross Validation and Pipelinding for RandomForest

In [ ]:
# Set up pipeline functions
pipeline = Pipeline([
    ("mms", MinMaxScaler()),
    ("rfr", RandomForestRegressor())  
])

# Set up our random search parameters
forest_rand_parameters = {
            'rfr__bootstrap': [True],
            'rfr__max_depth': [100, None],
            'rfr__min_samples_leaf': [1, 2, 3],
            'rfr__min_samples_split': [2, 3, 4],
            'rfr__n_estimators': [100, 200, 300]}


# Now set up our random search
forest_rand_search = RandomizedSearchCV(pipeline, 
                                 param_distributions = forest_rand_parameters, 
                                 n_iter = 10, 
                                 cv = 2, 
                                 verbose=2, 
                                 random_state=42, 
                                 n_jobs = -1,
                                 scoring="neg_root_mean_squared_error")

# Fit the grid search
forest_rand_search.fit(train_inputs, train_targets)

# Print the best parameters and score
print("Best parameter found:", forest_rand_search.best_params_)
print("Mean CV score of the best parameter is:", forest_rand_search.best_score_)

### Results

Random Search 1: 

Fitting 2 folds for each of 20 candidates, totalling 40 fits, 16 fits failed due to memmory errors

Set up our random search parameters
forest_rand_parameters = {
            'rfr__bootstrap': [True, False],
            'rfr__max_depth': [10, 50, 100, None],
            'rfr__min_samples_leaf': [1, 4],
            'rfr__min_samples_split': [2, 10],
            'rfr__n_estimators': [200, 400, 600]}

Best parameter found: {'rfr__n_estimators': 200, 'rfr__min_samples_split': 2, 'rfr__min_samples_leaf': 4, 'rfr__max_depth': None, 'rfr__bootstrap': True}
Mean CV score of the best parameter is: -1366.8457776056557

Result Search 2: 

Fitting 2 folds for each of 10 candidates, totalling 20 fits
2 fits failed out of a total of 20

forest_rand_parameters = {
            'rfr__bootstrap': [True],
            'rfr__max_depth': [100, None],
            'rfr__min_samples_leaf': [1, 2, 3],
            'rfr__min_samples_split': [2, 3, 4],
            'rfr__n_estimators': [100, 200, 300]}

Best parameter found: {'rfr__n_estimators': 300, 'rfr__min_samples_split': 3, 'rfr__min_samples_leaf': 1, 'rfr__max_depth': None, 'rfr__bootstrap': True}
Mean CV score of the best parameter is: -1288.8713318532295

### Comparison Scoring

To compare with the XGBoost and improve the bredth of our scoring here we will use the identified best fit parameters but do 2 cross validation splits and use three metrics. I wanted to do more splits to improve accuaracy but 10 spits crashed my compute and even 2 took almost 40 minutes to process.

In [ ]:
# Set up pipeline functions
pipeline = Pipeline([
    ("mms", MinMaxScaler()),
    ("rfr", RandomForestRegressor())  
])

#Set scoring functions dictionary
scoring_metrics= {"RMSE": "neg_root_mean_squared_error","MAE":"neg_mean_absolute_error","r2": "r2"}

# Set up our random search parameters
forest_rand_parameters = {
            'rfr__bootstrap': [True],
            'rfr__max_depth': [None],
            'rfr__min_samples_leaf': [1],
            'rfr__min_samples_split': [3],
            'rfr__n_estimators': [300]}


# Now set up our random search
forest_rand_search = RandomizedSearchCV(pipeline, 
                                 param_distributions = forest_rand_parameters, 
                                 n_iter = 10, 
                                 cv = 2, 
                                 verbose=2, 
                                 random_state=42, 
                                 n_jobs = -1,
                                 scoring= scoring_metrics,
                                 refit="RMSE")


# Fit the grid search
forest_rand_search.fit(train_inputs, train_targets)

# Print the best parameters and score
print("Best parameter found:", forest_rand_search.best_params_)
print("Mean CV score of the best parameter is:", forest_rand_search.best_score_)

In [ ]:
print(forest_rand_search.cv_results_)

Results:
      
       'split0_test_RMSE': array([-1262.65322262]), 
       'split1_test_RMSE': array([-1318.25736171]), 
       'mean_test_RMSE': array([-1290.45529217]), 
       'std_test_RMSE': array([27.80206955]), 
       'rank_test_RMSE': array([1]), 
       'split0_test_MAE': array([-823.3020933]), 
       'split1_test_MAE': array([-873.72220812]), 
       'mean_test_MAE': array([-848.51215071]), 
       'std_test_MAE': array([25.21005741]), 
       'rank_test_MAE': array([1]), 
       'split0_test_r2': array([0.83627383]), 
       'split1_test_r2': array([0.81646679]), 
       'mean_test_r2': array([0.82637031]), 
       'std_test_r2': array([0.00990352]), 
       'rank_test_r2': array([1])}



# MLP Model

## Simple model run

Same as above, this was just used to test parameters to see how long run time would be.

In [ ]:
MLP = MLPRegressor(solver='lbfgs', random_state=44, hidden_layer_sizes=(5, 5))
MLP.fit(X_train,y_train)


MLPRegressor(hidden_layer_sizes=(5, 5), random_state=44, solver='lbfgs')

### RMSE and RMPSE for Simple Model

In [ ]:
train_preds = MLP.predict(X_train)
test_preds = MLP.predict(X_test)

    # Get RMSE
train_rmse = np.round(root_mean_squared_error(y_train, train_preds, squared=False), 5)
val_rmse = np.round(root_mean_squared_error(y_test, test_preds, squared=False), 5)

    # Get RMSPE
train_rmspe = np.round(rmspe(y_train, train_preds), 5)
val_rmspe = np.round(rmspe(y_test, test_preds), 5)


print(f"Train RMSE: {train_rmse}")
print(f"Val RMSE: {val_rmse}")
print()
print(f"Train RMSPE: {train_rmspe}")
print(f"Val RMSPE: {val_rmspe}")

### Results Record

Results: 

Base Run: solver='adam', random_state=44, hidden_layer_sizes=(3,3)
Train RMSE: 2765.3078
Val RMSE: 2767.05624

Train RMSPE: 0.50719
Val RMSPE: 0.59677
1m 31s

Best parameter found: 

random search, iteration=5, CV=2

Parameters:

'mlp__hidden_layer_sizes': [(5,), (10,), (3, 3), (5, 5)],
'mlp__solver': ["adam", "sgd"]

Best: 'mlp__hidden_layer_sizes': (10,), 'mlp__solver': 'adam'

Train RMSE: 2765.31153
Val RMSE: 2767.08846

Train RMSPE: 0.50546
Val RMSPE: 0.5947

## MLP Cross Validation and Grid Search Pipeline

### Grid Search

In [ ]:
# Set up pipeline functions
pipeline = Pipeline([
    ("mms", MinMaxScaler()),
    ("mlp", MLPRegressor(max_iter=5000))  
])

# Set up our grid search parameters
gs_parameters = {
    'mlp__hidden_layer_sizes': [(10,10), (20,20),(50,50),(100,100)],
    'mlp__solver': ["lbfgs", "sgd"],
    "mlp__alpha": [.0001, .005]
}

# Now set up our grid search
grid_search = GridSearchCV(pipeline,
                           param_grid=gs_parameters,
                           cv=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

# Fit the grid search
grid_search.fit(train_inputs, train_targets)

# Print the best parameters and score
print("Best parameter found:", grid_search.best_params_)
print("Mean CV score of the best parameter is:", grid_search.best_score_)

1st Result: 

11m18s
    'mlp__hidden_layer_sizes': [(5,), (10,), (3, 3), (5, 5)],
    'mlp__solver': ["adam", "lbfgs", "sgd"]
Best parameter found: {'mlp__hidden_layer_sizes': (5, 5), 'mlp__solver': 'lbfgs'}
Mean CV score of the best parameter is: -2725.7188381885094

2nd Result: 
    'mlp__hidden_layer_sizes': [(10,10), (20,20),(50,50),(100,100)],
    'mlp__solver': ["lbfgs", "sgd"],
    "mlp__alpha": [.0001, .005]
35m5s, max itr=1000
Best parameter found: {'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (10, 10), 'mlp__solver': 'lbfgs'}
Mean CV score of the best parameter is: -2702.6066369004

### Random Search

In [ ]:
# Set up pipeline functions
pipeline = Pipeline([
    ("ss", preprocessing.StandardScaler()),
    ("mlp", MLPRegressor(max_iter=1000))  
])

# Set up our grid search parameters
rs_parameters = {
    'mlp__hidden_layer_sizes': [(5,), (10,), (3, 3), (5, 5)],
    'mlp__solver': ["adam", "sgd"]
}

# Now set up our grid search
random_search = RandomizedSearchCV(pipeline,
                           param_grid=rs_parameters,
                           cv=3,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           n_iter = 5)

# Fit the grid search
random_search.fit(train_inputs, train_targets)

# Print the best parameters and score
print("Best parameter found:", random_search.best_params_)
print("Mean CV score of the best parameter is:", random_search.best_score_)

### Results

Random Search 1: 
Set up our random search parameters
rs_parameters = {
    'mlp__hidden_layer_sizes': [(5,), (10,), (3, 3), (5, 5)],
    'mlp__solver': ["adam", "sgd"]
}

Now set up our random search
random_search = RandomizedSearchCV(pipeline,
                           param_grid=rs_parameters,
                           cv=3,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           n_iter = 5)

Best parameter found: {'mlp__hidden_layer_sizes': (10,), 'mlp__solver': 'adam'}
Mean CV score of the best parameter is: -7303543.3243916

# Forcasting Sales with RandomForest

Since I am training the model on the whole data set, am going to scale all of the imputs.

### Process in whole training set

In [14]:
# Scaling

# Encoded variables: train_inputs, train_targets
final_train=train_inputs

#Train our MinMaxScaler on the colums we want to scale
scaler = MinMaxScaler().fit(final_train[num_cols])

# Now use the scaler to scale all of the colums we have trained it on.
final_train[num_cols] = scaler.transform(final_train[num_cols])


In [16]:
# Train function 
random_forest_final = RandomForestRegressor(random_state=18,
                                            n_estimators= 300,
                                            min_samples_split= 3, 
                                            min_samples_leaf= 1,
                                            max_depth= None, 
                                            bootstrap= True)

random_forest_final.fit(final_train,train_targets)

RandomForestRegressor(min_samples_split=3, n_estimators=300, random_state=18)

### Save the model

In [57]:
joblib.dump(random_forest_final, "random_forest.z",compress=True)

['random_forest.z']

### Check Final Model Performance

In [ ]:
random_forest_load=joblib.load("random_forest.z")

We did not include this model file in our report since it was not our final model, and the file, even compressed was over 2GB.

In [ ]:
train_preds = random_forest_final.predict(X_train)
test_preds = random_forest_final.predict(X_test)

In [ ]:
train_rmse = np.round(root_mean_squared_error(y_train, train_preds), 5)
val_rmse = np.round(root_mean_squared_error(y_test, test_preds), 5)

    # Get RMSPE
train_rmspe = np.round(rmspe(y_train, train_preds), 5)
val_rmspe = np.round(rmspe(y_test, test_preds), 5)


print(f"Train RMSE: {train_rmse}")
print(f"Val RMSE: {val_rmse}")
print()
print(f"Train RMSPE: {train_rmspe}")
print(f"Val RMSPE: {val_rmspe}")

Since this is performed on all the training data it does not reflect the models overall performace. But I wanted to run it, just to make sure the training worked.

Train RMSE: 451.6194
Val RMSE: 450.25315

Train RMSPE: 0.08873
Val RMSPE: 0.14028

### Pre-Process Test Data

In [25]:
# Test Data Variable: test_inputs
#Already encoded so I will just need to scale the data before running the model on it.

#Train our MinMaxScaler on the colums we want to scale
scaler = MinMaxScaler().fit(test_inputs[num_cols])

# Now use the scaler to scale all of the colums we have trained it on.
test_inputs[num_cols] = scaler.transform(test_inputs[num_cols])


### Run Predict 

In [26]:
yhat=random_forest_final.predict(test_inputs)

In [27]:
pd.DataFrame(yhat).to_csv("forest_output.csv", index = None)

In [36]:
Store_open_test.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,4,2015-09-17,7802.021448,NaN,1.0,1,0,0,c,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,9,17
1,3,4,2015-09-17,11136.705849,NaN,1.0,1,0,0,a,...,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,9,17
2,7,4,2015-09-17,12683.320905,NaN,1.0,1,0,0,a,...,24000.0,4.0,2013.0,0,NaN,NaN,NaN,2015,9,17
3,8,4,2015-09-17,8242.953381,NaN,1.0,1,0,0,a,...,7520.0,10.0,2014.0,0,NaN,NaN,NaN,2015,9,17
4,9,4,2015-09-17,9247.781849,NaN,1.0,1,0,0,a,...,2030.0,8.0,2000.0,0,NaN,NaN,NaN,2015,9,17


In [41]:
print(yhat)

[ 7802.02144841 11136.70584921 12683.32090476 ...  5949.15116667
 20875.00146032  5400.97959524]


In [ ]:
yhat=np.array(yhat)
Store_open_test['Sales']=round(yhat,2)

In [45]:
Store_open_test = Store_open_test.sort_values(by='Date')
Store_open_test.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
41087,1115,6,2015-08-01,5400.979595,NaN,1.0,0,0,1,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,8,1
41085,1113,6,2015-08-01,5949.151167,NaN,1.0,0,0,0,a,...,9260.0,NaN,NaN,0,NaN,NaN,NaN,2015,8,1
41084,1112,6,2015-08-01,11073.642067,NaN,1.0,0,0,0,c,...,1880.0,4.0,2006.0,0,NaN,NaN,NaN,2015,8,1
41083,1111,6,2015-08-01,3422.162175,NaN,1.0,0,0,0,a,...,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",2015,8,1
41082,1109,6,2015-08-01,3732.518905,NaN,1.0,0,0,0,c,...,3490.0,4.0,2011.0,1,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,1


In [48]:
pd.DataFrame(Store_open_test).to_csv("random_forest_results.csv", index = None)